In [4]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from datetime import datetime, timedelta, date


from gtda.time_series import TakensEmbedding
from gtda.time_series import SlidingWindow
from gtda.time_series import SingleTakensEmbedding

from gtda.time_series import PearsonDissimilarity
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Amplitude
from gtda.diagrams import PersistenceLandscape


from gtda.pipeline import make_pipeline


from sklearn import set_config
set_config(display='diagram')  # For HTML representations of pipelines

from sklearn.cluster import KMeans


import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import config as c 

key, secret = c.apis[0][0]+"a", c.apis[0][1]
client = Client(key, secret)
# config has variable 'apis = [[# YOUR API KEY],[#YOUR API SECRET]]

In [41]:
def get_date(timestamp):
    obj = datetime.fromtimestamp(timestamp/1000.0)
    return "%s/%s/%s %s:%s" % (obj.month, obj.day, obj.year, obj.hour, obj.minute)

def get_timestamp(date):
    dt_obj = datetime.strptime(date, '%m/%d/%Y %H:%M')
    return int(dt_obj.timestamp() * 1000)

def generate_volumebars(trades, frequency=10):
    times = trades[:,0]
    prices = trades[:,1]
    volumes = trades[:,2]
    ans = np.zeros(shape=(len(prices), 6))
    candle_counter = 0
    vol = 0
    lasti = 0
    for i in range(len(prices)):
        vol += volumes[i]
        if vol >= frequency:
            ans[candle_counter][0] = times[i]              # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])        # volume
            candle_counter += 1
            lasti = i+1
            vol = 0
    return ans[:candle_counter]

In [42]:
# [
#   {
#     "a": 26129,         // Aggregate tradeId
#     "p": "0.01633102",  // Price
#     "q": "4.70443515",  // Quantity
#     "f": 27781,         // First tradeId
#     "l": 27781,         // Last tradeId
#     "T": 1498793709153, // Timestamp
#     "m": true,          // Was the buyer the maker?
#     "M": true           // Was the trade the best price match?
#   }
# ]
orderflow = client.get_aggregate_trades(symbol="BTCUSDT", startTime = get_timestamp('8/25/2021 5:55'), endTime = get_timestamp('8/25/2021 6:55'))

trades = np.array([[float(i["T"]), float(i["p"]), float(i["q"])] for i in orderflow])

bars_arr = generate_volumebars(trades)

bars = pd.DataFrame([i[1:] for i in bars_arr], columns = ["Open", "High", "Low", "Close", "Volume"], index=pd.to_datetime([i[0] for i in bars_arr], unit='ms'))

In [56]:
fig = go.Figure(data=[go.Candlestick(x=bars.index,
                open=bars['Open'],
                high=bars['High'],
                low=bars['Low'],
                close=bars['Close'])])

fig.show()